## Plotting time series in ARTop

Time series in ARTop are based on Pandas DataFrames (https://pandas.pydata.org/) which are highly portable. Here we present some examples of the time series functionality in ARTop.


The time series variables in ARTop have the names:

```
windCur   : current-carrying part of winding rate (km^4/s)
helCur    : current-carrying part of field line helicity rate (Gauss^2km^4/s)
windPot   : potential part of winding rate (km^4/s)
helPot    : potential part of helicity rate (Gauss^2km^4/s)
windBraid : braiding component of winding rate (km^4/s)
helBraid  : braiding component of helicity rate (Gauss^2km^4/s)
wind      : total winding rate (km^4/s)
hel       : total helicity rate (Gauss^2km^4/s)
dL        : delta winding rate (km^$/s)
dH        : delta helicity rate (Gauss^2km^4/s)
```

These are stored in a Pandas DataFrame, and each variable is accessed using one of the above keys. 


For time-integrated versions of these variables (see Example 1), `_int` is added to every variable name.

For running mean and envelope calculations (see Example 2), `_ms` is added to every variable name.

Example variables are included below, but you should adapt them for the data you have.

In [ ]:
# Example 1: Plot the time series of a time-integrated quantity
current
import os
import sys
cwd = os.getcwd()
sys.path.insert(1, os.path.join(cwd,'source','python'))    # If your version of analysis.py is in another location, change this path.
from analysis import *

# Get the data.
outputDir = os.path.join(cwd,'output')                                        # Set the output directory (containing the Data folder and *.txt).
raw_series = readSeries(outputDir)                                            # Load the time series of all the basic ARTop time series variables.
accum = integrate_series(raw_series)                                          # Create accumulations of the raw variables, i.e. integrate in time.
time = accum['times(seconds)']                                                # Get the times (in seconds) from the relevant DataFrame.


# Plot the data
var = [accum['dL_int']]                                                       # Choose the variable to be plotted.
colour = ['r-']                                                               # The colour of the time series.
label = ['$\delta$L']                                                         # Set the label (variables that begin with an underscore will not be shown).
y_label = r'$\delta$L'+' '+ '$(km^4)$'                                        # Set the label of the y-axis.
title = ' '                                                                   # Set the title. This must be set but put ' ' if no title is wanted. 

plot(time, var, colour,label,y_label,title,unit='h')                          # Plot the time series with time in hours on the x-axis.
plt.show()

In [ ]:
# Example 2: Overplot a time series with a running mean plus envelope and flare times.

import os
import sys
cwd = os.getcwd()
sys.path.insert(1, os.path.join(cwd,'source','python'))    # If your version of analysis.py is in another location, change this path.
from analysis import *

# Get the time series data.
outputDir = os.path.join(cwd,'output')                                             # Set the output directory (containing the Data folder and *.txt).
raw_series = readSeries(outputDir)                                                 # Load the time series of all the basic ARTop time series variables.
offset = 3                                                                         # The offset (in time dumps) for calculating the running mean.  
num_std = 3                                                                        # The height (number of standard deviations) of the envelope above the running mean.
mean = running_mean_std(raw_series,offset,0)                                       # Calculate the running mean.
env = running_mean_std(raw_series,offset,num_std)                                  # Calculate the envelope. 

# Get the flare times from NOAA reports.
start_time = '2011-02-10 10:00:00'                                                 # Choose the start time.
end_time = '2011-02-19 06:00:00'                                                   # Choose the end time.
variable = 'FLA'                                                                   # Choose the variable to read (FLA for flare, XRA for X-ray) 
activeregion = 'AR11158'                                                           # Set the NOAA active region number. 
time_col = 'Begin'                                                                 # Select the Begin, Max or End. 

report = flare_times(start_time,end_time)                                          # Download the NOAA reports from the specified period.

                       
FL = report.toseconds(activeregion, variable, time_col)                            # Get the flare times in seconds (for overplotting). Note that the start time corresponds to t=0.

# Plot the data

time = raw_series['times(seconds)']                                                # Get the times (in seconds).
overplot_variables = [raw_series['dL'], mean['dL_ms'], env['dL_ms']]               # List the variables to be overplotted.   
colour = ['c-','r-','b:']                                                          # Select colours for the different curves.
label = ['$\dot{\delta L}$', '$\mu(\dot{\delta L})$', '$\mu+3\sigma$']             # Label the time series variables
y_label = '$\dot{\delta L}$ (km$^4$/s)'                                            # Label the y-axis.
title = ' '                                                                        # Set a plot title 
plot(time, overplot_variables, colour,label,y_label, title ,unit='h',oplot=[FL])


In [ ]:
# Example 3: Displaying flare reports

# This example shows what can be read from the NOAA reports and how it is structured for plotting within ARTop.

import os
import sys
cwd = os.getcwd()
sys.path.insert(1, os.path.join(cwd,'source','python'))    # If your version of analysis.py is in another location, change this path.
from analysis import *

# Get the flare times from NOAA reports.
start_time = '2011-02-10 10:00:00'                                                 # Choose the start time.
end_time = '2011-02-19 06:00:00'                                                   # Choose the end time.
variable1 = 'FLA'                                                                  # Choose the FLA flare time option. 
variable2 = 'XRA'                                                                  # Choose the GOES X-ray option.  
activeregion = 'AR11158'                                                           # Set the NOAA active region number. 
time_col = 'Begin'                                                                 # Select the Begin, Max or End. 

report = flare_times(start_time,end_time)                                          # Download the NOAA reports from the specified period.
FL = report.toseconds(activeregion, variable1, time_col)                           # Get the flare times in seconds.
XR = report.toseconds(activeregion, variable2, time_col)                           # Get the GOES X-ray times in seconds.

print(FL)                                                                          # Display the results. Colours correspond to how ARTop plots these quantities.  
print(XR)
# Note that the start time corresponds to t=0.